In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

sns.set_context("paper")
import matplotlib.ticker as ticker

from tqdm import tqdm # progress bars :)
tqdm.pandas()
from sklearn.metrics import r2_score
import scipy as sp

In [2]:
ep = pd.read_csv("gtdb-rs202.evolpaths.nucleotide.csv.gz")
ep

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,total_query_fragments,EzAAIm AAI,EzAAIm anchor CDS count,EzAAIm compare CDS count,EzAAIm matched CDS count,EzAAIm proteome coverage,EzAAIm id threshold,EzAAIm coverage threshold,minContain ANI,avgContain ANI
0,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1,0.155515,0.401210,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.926776,0.942109
1,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1000,0.160974,0.412247,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.928150,0.943416
2,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,2000,0.159965,0.417136,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.927531,0.943375
3,GCA_004524385.1_x_GCA_011364975.1,GCA_004524385.1,GCA_011364975.1,path1,family,nucleotide,21,1,0.000272,0.000828,...,NaN,0.631682,1965,3596,994,0.357490,0.4,0.5,0.689328,0.701284
4,GCA_004524385.1_x_GCA_011364975.1,GCA_004524385.1,GCA_011364975.1,path1,family,nucleotide,21,1000,0.000712,0.002187,...,NaN,0.631682,1965,3596,994,0.357490,0.4,0.5,0.721482,0.734246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221125,GCA_002409405.1_x_GCA_003567135.1,GCA_002409405.1,GCA_003567135.1,path999,phylum,nucleotide,51,1000,0.000000,0.000000,...,NaN,0.507298,3045,2723,351,0.121706,0.4,0.5,NaN,NaN
221126,GCA_002409405.1_x_GCA_003567135.1,GCA_002409405.1,GCA_003567135.1,path999,phylum,nucleotide,51,2000,0.000000,0.000000,...,NaN,0.507298,3045,2723,351,0.121706,0.4,0.5,NaN,NaN
221127,GCA_002409405.1_x_GCF_004319585.1,GCA_002409405.1,GCF_004319585.1,path999,superkingdom,nucleotide,51,1,0.000000,0.000000,...,NaN,0.485674,3045,3603,264,0.079422,0.4,0.5,NaN,NaN
221128,GCA_002409405.1_x_GCF_004319585.1,GCA_002409405.1,GCF_004319585.1,path999,superkingdom,nucleotide,51,1000,0.000000,0.000000,...,NaN,0.485674,3045,3603,264,0.079422,0.4,0.5,NaN,NaN


In [3]:
dna_compare = ep[ep["ANIb ident"].notna()] # only grab comparisons we can work with
dna_compare

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,total_query_fragments,EzAAIm AAI,EzAAIm anchor CDS count,EzAAIm compare CDS count,EzAAIm matched CDS count,EzAAIm proteome coverage,EzAAIm id threshold,EzAAIm coverage threshold,minContain ANI,avgContain ANI
0,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1,0.155515,0.401210,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.926776,0.942109
1,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1000,0.160974,0.412247,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.928150,0.943416
2,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,2000,0.159965,0.417136,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.927531,0.943375
18,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,31,1,0.117970,0.315380,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.942327,0.952893
19,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,31,1000,0.115089,0.307900,...,487.0,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.941682,0.952198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221095,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,31,1000,0.025155,0.052790,...,1130.0,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.905350,0.907414
221096,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,31,2000,0.027450,0.056604,...,1130.0,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.908234,0.909880
221112,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,51,1,0.011440,0.023716,...,1130.0,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.927581,0.928421
221113,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,51,1000,0.011678,0.024665,...,1130.0,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.927643,0.928810


In [4]:
def fmh_ani_underestimate(row):
    row["ac_under"] = False
    anib = float(row["ANIb ident"])
    acANI = float(row["avgContain ANI"])
    if acANI <= anib:
        row["ac_under"] = True
    return row

In [5]:
dna_compare = dna_compare.progress_apply(fmh_ani_underestimate, axis=1)
dna_compare

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30087/30087 [00:23<00:00, 1295.22it/s]


,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,EzAAIm AAI,EzAAIm anchor CDS count,EzAAIm compare CDS count,EzAAIm matched CDS count,EzAAIm proteome coverage,EzAAIm id threshold,EzAAIm coverage threshold,minContain ANI,avgContain ANI,ac_under
0,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1,0.155515,0.401210,...,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.926776,0.942109,False
1,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,1000,0.160974,0.412247,...,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.928150,0.943416,False
2,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,21,2000,0.159965,0.417136,...,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.927531,0.943375,False
18,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,31,1,0.117970,0.315380,...,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.942327,0.952893,False
19,GCA_004524385.1_x_GCA_004524535.1,GCA_004524385.1,GCA_004524535.1,path1,genus,nucleotide,31,1000,0.115089,0.307900,...,0.929069,1965,3590,1495,0.538254,0.4,0.5,0.941682,0.952198,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221095,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,31,1000,0.025155,0.052790,...,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.905350,0.907414,False
221096,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,31,2000,0.027450,0.056604,...,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.908234,0.909880,False
221112,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,51,1,0.011440,0.023716,...,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.927581,0.928421,False
221113,GCA_002409405.1_x_GCF_011030385.1,GCA_002409405.1,GCF_011030385.1,path999,genus,nucleotide,51,1000,0.011678,0.024665,...,0.917134,3045,3255,2466,0.782857,0.4,0.5,0.927643,0.928810,False


In [24]:
underestimatedANI = dna_compare[dna_compare["ac_under"] ==True]
not_underestimatedANI = dna_compare[dna_compare["ac_under"] ==False]
underestimatedANI

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,EzAAIm AAI,EzAAIm anchor CDS count,EzAAIm compare CDS count,EzAAIm matched CDS count,EzAAIm proteome coverage,EzAAIm id threshold,EzAAIm coverage threshold,minContain ANI,avgContain ANI,ac_under
54,GCA_002255135.1_x_GCA_002254825.2,GCA_002255135.1,GCA_002254825.2,path10,genus,nucleotide,21,1,0.471551,0.642012,...,0.984206,1255,1159,687,0.569180,0.4,0.5,0.978956,0.979037,True
55,GCA_002255135.1_x_GCA_002254825.2,GCA_002255135.1,GCA_002254825.2,path10,genus,nucleotide,21,1000,0.447260,0.618371,...,0.984206,1255,1159,687,0.569180,0.4,0.5,0.977327,0.977349,True
56,GCA_002255135.1_x_GCA_002254825.2,GCA_002255135.1,GCA_002254825.2,path10,genus,nucleotide,21,2000,0.434663,0.617188,...,0.984206,1255,1159,687,0.569180,0.4,0.5,0.975588,0.976435,True
72,GCA_002255135.1_x_GCA_002254825.2,GCA_002255135.1,GCA_002254825.2,path10,genus,nucleotide,31,1,0.462011,0.633262,...,0.984206,1255,1159,687,0.569180,0.4,0.5,0.985246,0.985308,True
73,GCA_002255135.1_x_GCA_002254825.2,GCA_002255135.1,GCA_002254825.2,path10,genus,nucleotide,31,1000,0.490196,0.662252,...,0.984206,1255,1159,687,0.569180,0.4,0.5,0.986375,0.986585,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217784,GCA_002292365.1_x_GCA_902605755.1,GCA_002292365.1,GCA_902605755.1,path943,genus,nucleotide,21,2000,0.003697,0.009357,...,0.824654,2285,1492,1041,0.551231,0.4,0.5,0.784246,0.792397,True
220216,GCA_002728175.1_x_GCA_014384505.1,GCA_002728175.1,GCA_014384505.1,path984,family,nucleotide,21,1000,0.000328,0.000736,...,0.640743,1625,1357,925,0.620389,0.4,0.5,0.701965,0.705607,True
220320,GCA_902587125.1_x_GCA_902597195.1,GCA_902587125.1,GCA_902597195.1,path986,genus,nucleotide,21,1,0.140095,0.358310,...,0.953517,1364,778,654,0.610644,0.4,0.5,0.923268,0.937784,True
220321,GCA_902587125.1_x_GCA_902597195.1,GCA_902587125.1,GCA_902597195.1,path986,genus,nucleotide,21,1000,0.138902,0.364796,...,0.953517,1364,778,654,0.610644,0.4,0.5,0.922365,0.937740,True


In [8]:
underestimatedANI.columns

Index(['comparison_name', 'anchor_name', 'compare_name', 'path',
       'lowest_common_rank', 'alphabet', 'ksize', 'scaled', 'jaccard',
       'max_containment', 'anchor_containment', 'compare_containment',
       'anchor_hashes', 'compare_hashes', 'num_common', 'jaccard ANI',
       'jani_low', 'jani_high', 'containA ANI', 'containAani_low',
       'containAani_high', 'containB ANI', 'containBani_low',
       'containBani_high', 'maxContain ANI', 'mcANI_low', 'mcANI_high',
       'jaccard_warning', 'alpha-ksize', 'compareM AAI', 'compareM_std_aai',
       'genes_in_anchor', 'genes_in_ref', 'orthologous_genes',
       'orthologous_fraction', 'ANIb ident', 'ANIb_coverage',
       'ANIb_aln_length', 'ANIb_sim_errors', 'ANIb_hadamard', 'ANIm ident',
       'ANIm_coverage', 'ANIm_aln_length', 'ANIm_sim_errors', 'ANIm_hadamard',
       'fastani ANI', 'num_bidirectional_fragment_mappings',
       'total_query_fragments', 'EzAAIm AAI', 'EzAAIm anchor CDS count',
       'EzAAIm compare CDS cou

In [48]:
cols_of_interest = ["avgContain ANI", "maxContain ANI", "ANIb ident", "ANIb_aln_length", 
                    "ANIb_sim_errors", "ANIb_coverage", "num_common", 
                    "anchor_hashes", "compare_hashes", "lowest_common_rank"]
drop_cols = [ x for x in list(underestimatedANI.columns) if x not in cols_of_interest ]
drop_cols

['comparison_name',
 'anchor_name',
 'compare_name',
 'path',
 'alphabet',
 'ksize',
 'scaled',
 'jaccard',
 'max_containment',
 'anchor_containment',
 'compare_containment',
 'jaccard ANI',
 'jani_low',
 'jani_high',
 'containA ANI',
 'containAani_low',
 'containAani_high',
 'containB ANI',
 'containBani_low',
 'containBani_high',
 'mcANI_low',
 'mcANI_high',
 'jaccard_warning',
 'alpha-ksize',
 'compareM AAI',
 'compareM_std_aai',
 'genes_in_anchor',
 'genes_in_ref',
 'orthologous_genes',
 'orthologous_fraction',
 'ANIb_hadamard',
 'ANIm ident',
 'ANIm_coverage',
 'ANIm_aln_length',
 'ANIm_sim_errors',
 'ANIm_hadamard',
 'fastani ANI',
 'num_bidirectional_fragment_mappings',
 'total_query_fragments',
 'EzAAIm AAI',
 'EzAAIm anchor CDS count',
 'EzAAIm compare CDS count',
 'EzAAIm matched CDS count',
 'EzAAIm proteome coverage',
 'EzAAIm id threshold',
 'EzAAIm coverage threshold',
 'minContain ANI',
 'ac_under']

In [36]:
underestimatedANI_k21_scaled1 = underestimatedANI[(underestimatedANI["ksize"] == 21) & (underestimatedANI["scaled"] == 1)]
not_underestimatedANI_k21_scaled1 = not_underestimatedANI[(not_underestimatedANI["ksize"] == 21) & (not_underestimatedANI["scaled"] == 1)]

In [17]:
underestimatedANI_k21_scaled1[cols_of_interest].describe()

,avgContain ANI,ANIb ident,ANIb_aln_length,ANIb_sim_errors,ANIb_coverage,ANIb_hadamard,num_common,anchor_hashes,compare_hashes
count,65.000000,65.000000,6.500000e+01,65.000000,65.000000,65.000000,6.500000e+01,6.500000e+01,6.500000e+01
mean,0.929084,0.939960,1.112423e+06,60338.492308,0.532368,0.503200,6.253669e+05,2.069631e+06,2.136121e+06
std,0.049173,0.054081,5.949622e+05,52549.436783,0.103327,0.110809,4.896310e+05,1.151412e+06,1.061077e+06
min,0.775116,0.777602,3.246330e+05,1129.500000,0.300427,0.233637,6.041000e+03,6.038890e+05,5.287540e+05
25%,0.916556,0.925402,6.584030e+05,14458.000000,0.463692,0.439486,2.042650e+05,1.183270e+06,1.217324e+06
50%,0.943571,0.949239,9.510000e+05,47831.500000,0.529505,0.497562,5.286960e+05,1.720962e+06,1.981438e+06
75%,0.962009,0.984518,1.490529e+06,92140.500000,0.594820,0.585306,9.394310e+05,2.768436e+06,2.703421e+06
max,0.981345,0.998381,2.756130e+06,233727.000000,0.759995,0.722450,1.868903e+06,5.632306e+06,5.243505e+06


In [25]:
not_underestimatedANI[cols_of_interest].describe()

,avgContain ANI,ANIb ident,ANIb_aln_length,ANIb_sim_errors,ANIb_coverage,ANIb_hadamard,num_common,anchor_hashes,compare_hashes
count,28071.000000,29641.000000,2.964100e+04,2.964100e+04,29641.000000,29641.000000,2.964100e+04,2.964100e+04,2.964100e+04
mean,0.863790,0.799063,1.663809e+06,3.095077e+05,0.509600,0.413359,3.779751e+04,1.098006e+06,1.097645e+06
std,0.050432,0.059125,1.008189e+06,1.556108e+05,0.145569,0.143312,1.556325e+05,1.839782e+06,1.845743e+06
min,0.707006,0.702407,1.189910e+05,1.010700e+04,0.300261,0.214946,0.000000e+00,9.700000e+01,1.050000e+02
25%,0.827137,0.753802,9.811280e+05,1.983760e+05,0.382473,0.291988,9.000000e+00,1.698000e+03,1.691000e+03
50%,0.866639,0.783257,1.447066e+06,2.902345e+05,0.489695,0.384935,6.900000e+01,3.267000e+03,3.251000e+03
75%,0.899792,0.834303,2.104678e+06,3.985350e+05,0.620031,0.513736,6.054000e+03,2.027812e+06,2.013657e+06
max,0.990983,0.990523,8.640779e+06,1.315955e+06,0.948156,0.897673,4.030873e+06,1.455804e+07,1.379042e+07


In [23]:
underestimatedANI_k21_scaled1.drop(columns=drop_cols)

,lowest_common_rank,anchor_hashes,compare_hashes,num_common,ANIb ident,ANIb_coverage,ANIb_aln_length,ANIb_sim_errors,ANIb_hadamard,avgContain ANI
54,genus,1070143,1066409,684647,0.992356,0.560973,601844.0,4366.0,0.556684,0.979037
2214,genus,1270446,1077129,381997,0.948531,0.683559,798364.5,41047.5,0.648372,0.948108
8964,genus,1479041,1124824,378078,0.950058,0.749391,1013336.0,50297.0,0.711917,0.943258
12042,genus,1789822,1702577,763445,0.978550,0.515151,911390.0,19245.5,0.504099,0.961383
12312,genus,3206822,3716494,991825,0.946601,0.490062,1702676.0,89345.0,0.463967,0.942343
...,...,...,...,...,...,...,...,...,...,...
202122,genus,2612951,3272408,1392052,0.988212,0.444836,1313290.0,14458.0,0.439585,0.965287
203202,genus,808289,1124891,176427,0.925402,0.653102,620164.0,45975.5,0.604414,0.922825
207738,genus,2533142,2078592,967694,0.975962,0.529505,1215641.0,28464.5,0.516719,0.959729
217566,genus,3744781,3839508,736805,0.927135,0.461608,1794364.0,128629.0,0.427980,0.924952


In [59]:
underestimatedANI_k21_scaled1[underestimatedANI_k21_scaled1["ANIb ident"] >= 0.95]

,comparison_name,anchor_name,compare_name,path,lowest_common_rank,alphabet,ksize,scaled,jaccard,max_containment,...,EzAAIm AAI,EzAAIm anchor CDS count,EzAAIm compare CDS count,EzAAIm matched CDS count,EzAAIm proteome coverage,EzAAIm id threshold,EzAAIm coverage threshold,minContain ANI,avgContain ANI,ac_under
54,GCA_002255135.1_x_GCA_002254825.2,GCA_002255135.1,GCA_002254825.2,path10,genus,nucleotide,21,1,0.471551,0.642012,...,0.984206,1255,1159,687,0.569180,0.4,0.5,0.978956,0.979037,True
8964,GCA_002692065.1_x_GCA_902551855.1,GCA_002692065.1,GCA_902551855.1,path1148,genus,nucleotide,21,1,0.169863,0.336122,...,0.952905,1435,1024,840,0.683205,0.4,0.5,0.937110,0.943258,True
12042,GCA_009649835.1_x_GCA_013374465.1,GCA_009649835.1,GCA_013374465.1,path12,genus,nucleotide,21,1,0.279757,0.448406,...,0.976038,1946,1916,1024,0.530295,0.4,0.5,0.960239,0.961383,True
22518,GCA_004377135.1_x_GCA_004376545.1,GCA_004377135.1,GCA_004376545.1,path1374,genus,nucleotide,21,1,0.358325,0.579282,...,0.978096,1288,1207,706,0.565932,0.4,0.5,0.966071,0.970204,True
23058,GCA_002313895.1_x_GCA_002710885.1,GCA_002313895.1,GCA_002710885.1,path1383,genus,nucleotide,21,1,0.210930,0.451112,...,0.931076,3400,2380,1839,0.636332,0.4,0.5,0.941780,0.952292,True
24300,GCA_009392775.1_x_GCA_002716645.1,GCA_009392775.1,GCA_002716645.1,path1403,genus,nucleotide,21,1,0.225362,0.458355,...,0.950768,1595,1055,587,0.443019,0.4,0.5,0.945342,0.954438,True
31320,GCA_014383215.1_x_GCA_013203195.1,GCA_014383215.1,GCA_013203195.1,path1520,genus,nucleotide,21,1,0.214772,0.422642,...,0.949317,4530,3390,2161,0.545707,0.4,0.5,0.944869,0.952344,True
43740,GCA_007123535.1_x_GCA_007117565.1,GCA_007123535.1,GCA_007117565.1,path1728,genus,nucleotide,21,1,0.308900,0.633340,...,0.987338,1055,594,355,0.430564,0.4,0.5,0.954509,0.966497,True
56646,GCA_002007285.1_x_GCA_001983475.1,GCA_002007285.1,GCA_001983475.1,path1943,genus,nucleotide,21,1,0.505841,0.719268,...,0.990991,2261,1932,1326,0.632483,0.4,0.5,0.978259,0.981345,True
58212,GCA_002730735.1_x_GCA_002720285.1,GCA_002730735.1,GCA_002720285.1,path197,genus,nucleotide,21,1,0.142469,0.272181,...,0.920458,1385,1406,664,0.475815,0.4,0.5,0.932437,0.936176,True


In [55]:
underestimatedANI_k21_scaled1[underestimatedANI_k21_scaled1["ANIb ident"] >= 0.97].drop(columns=drop_cols)

,lowest_common_rank,anchor_hashes,compare_hashes,num_common,maxContain ANI,ANIb ident,ANIb_coverage,ANIb_aln_length,ANIb_sim_errors,avgContain ANI
54,genus,1070143,1066409,684647,0.979119,0.992356,0.560973,601844.0,4366.0,0.979037
12042,genus,1789822,1702577,763445,0.962527,0.978550,0.515151,911390.0,19245.5,0.961383
22518,genus,1233849,1031717,597655,0.974337,0.984221,0.594820,670506.5,10107.0,0.970204
24300,genus,1479551,991512,454464,0.963533,0.984518,0.442293,541341.0,7925.5,0.954438
43740,genus,890236,528754,334881,0.978485,0.996284,0.495393,329709.5,1129.5,0.966497
56646,genus,2376822,2082750,1498056,0.984431,0.990523,0.651801,1490529.0,12876.0,0.981345
82620,genus,1123526,1498203,622352,0.972262,0.986703,0.504263,650590.5,8350.5,0.965646
84348,genus,1103748,1217324,531569,0.965806,0.989452,0.443262,517128.0,5090.0,0.963559
88236,genus,1907868,1952563,1278954,0.981135,0.997196,0.638585,1246868.0,3388.5,0.980595
90828,genus,2818349,2363846,1401974,0.975430,0.992837,0.550000,1431984.0,9354.5,0.971363


In [47]:
underestimatedANI_k21_scaled1[underestimatedANI_k21_scaled1["ANIb ident"] >= 0.95]["ANIb_coverage"].mean()

0.5600772017571054

In [46]:
underestimatedANI_k21_scaled1[underestimatedANI_k21_scaled1["ANIb ident"] >= 0.95]["ANIb_coverage"].median()

0.5537536854603687

In [35]:
underestimatedANI_k21_scaled1["ANIb_coverage"].mean()

0.5323682358832293

In [45]:
not_underestimatedANI_k21_scaled1[not_underestimatedANI_k21_scaled1["ANIb ident"] >= 0.95]["ANIb_coverage"].mean()

0.7458606151446665

In [50]:
# need to make this a percentage???
not_underestimatedANI_k21_scaled1[not_underestimatedANI_k21_scaled1["ANIb ident"] >= 0.95]["ANIb_aln_length"].mean()

2604406.388888889

In [53]:
underestimatedANI_k21_scaled1[underestimatedANI_k21_scaled1["ANIb ident"] >= 0.95]["ANIb_aln_length"].mean()

1168818.9375